In [1]:
# CNN(Convolutional Neural Network)

# FC layer(Fully Connected layer)
# 이전 layer의 모든 node가 다음 layer의 모든 node에 연결되면 학습되는 layer구조
# FC layer를 다른 말로 Dense layer라고도 함

# 지금까지 우리가 작업한 신경망은 모두 FC layer를 이용하고 있음

# FC layer의 특징은 MNIST의 예제처럼 입력데이터가 1차원으로 한정됨
# 즉, 각각의 이미지가 1차원으로 표현되어야 함
# 그래서 2차원 이미지를 1차원으로 변환시켜서 사용한 것

# 우리가 사용한 MNIST예제는 상당히 간단한 이미지 학습, 예측 예제임

# 이미지 학습의 가장 큰 문제는
# 이미지가 살짝 휘어있거나 크기가 제각각이거나 변형이 조금만 생겨도 학습이 힘들어짐

# 이런 경우 training data가 굉정히 많이 필요하고 추가적으로 학습할 때 많은 시간을 요구하기 됨

#############################################

# 고민하면서 사람이 학습하는 방식 모델림 함
# 찾아낸 방법은 이미지의 픽셀값을 그대로 입력하는게 아닌
# 이 이미지를 대표하는 특징을 도출하여 신경망에 여러 개 넣어서 학습하는 방식

# 1장의 컬러사진은 width, height, color(depth) 3차원으로 표현
# 여러장의 사진이 사용되기에 입력데이터는 4차원으로 표현

# 실제 이미지 1장은 3차원이고 이걸 flatten시켜서 1차원으로 표현해야 함
# 크기를 조절해야 되기에 공간에 대한 데이터를 유실할 우려가 있음
# 이런 데이터 유실 때문에 학습과 예측에 문제가 발생하게 됨

# 공간데이터의 유실을 없어고, 이미지의 특성을 추출, 학습이 용이하게 만드는 방식 ==> CNN
# (하나의 원본이미지를 원본의 특성을 가지고 있는 홀더 혹은 이미지로 여러개 만든다)

# 3차원 데이터에 필터를 적용 => 적용될 필터도 3차원 필터 지님
# width, height은 아무렇게나 지정 가능하나 depth는 원본과 동일한 값을 지녀야 함


In [2]:
# code로 알아보자
# 사용되어지는 함수부터 알아봅세
# Sample CNN

import tensorflow as tf
import numpy as np

# 입력데이터의 형식: 3*3*1   이미지 1개 이용
# 입력데이터 => (이미지개수. width, height, color) => (1,3,3,1)
# 총 9개의 데이터가 사용(1~9)
image=np.array([[[[1],[2],[3]],
                 [[4],[5],[6]],
                 [[7],[8],[9]]]], dtype=np.float32)
print(f'image shape: {image.shape}')   # (1,3,3,1)

# Activation map을 위한 filter를 정의 (width, height, depth(color), filter 개수)
# filter (2,2,1,3) // (2,2,1) * 3
weight=np.array([[[[1,10,-1]],[[1,10,-1]]],
                 [[[1,10,-1]],[[1,10,-1]]]])
print(f'weight shape: {weight.shape}')   # (2,2,1,3)

# stride=1 (가로, 세로를 1씩 움직여요)
conv2d=tf.nn.conv2d(image, weight, strides=[1,1,1,1], padding='VALID')
# strides => 배열 맞춰주어야 함,
#            가로세로 크기 같게 맞출 것
#            [1] => 가로, [2] => 세로
#            [0],[3] => dummy
# padding='VALID' => padding X
#         'SAME'  => padding O
print(f'conv2d shape: {conv2d.shape}')   # (1, 2, 2, 3)
                                         # (img 개수, 가로, 세로, filter?)
sess=tf.Session()
conv2d=sess.run(conv2d)

# ReLu 들어감

# pooling layer
pool=tf.nn.max_pool(conv2d, ksize=[1,2,2,1],
                    strides=[1,1,1,1], padding='SAME')
# ksize => kernel size( [1],[2] )
#          [0],[3] => dummy
print(f'pool shape: {pool.shape}')



C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\envs\cpu_env\lib\site-packages\ten

image shape: (1, 3, 3, 1)
weight shape: (2, 2, 1, 3)
conv2d shape: (1, 2, 2, 3)
pool shape: (1, 2, 2, 3)


In [3]:
# Convolution 결과 이미지가 원본 이미지에 비해 어떻게 다른가!?
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

# Data Loading
mnist=input_data.read_data_sets('./data/mnist', one_hot=True)

img=mnist.train.images[39512].reshape(28,28)
plt.imshow(img, cmap='Greys')

###############################################
# 해당 이미지 convolution 처리 해 보죠
# 입력데이터 => (이미지 개수, width, height, color) => (1,3,3,1)
img=img.reshape(-1,28,28,1)
print(f'img shape: {img.shape}')

# Activation map을 위한 filter 정의
# (width, height, color, filter 개수)
W=tf.Variable(tf.random_normal([3,3,1,50]), name='filter1')

conv2d=tf.nn.conv2d(img, W, strides=[1,2,2,1],padding='SAME')
print(f'conv2d shape: {conv2d.shape}')   # (1, 14, 14, 5)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
conv2d=sess.run(conv2d)

# 이미지를 표현하기 위하여 축을 전환
# (1,14,14,5) => (5,14,14,1)
conv2d_img=np.swapaxes(conv2d,0,3)
print(f'conv2d_img shape: {conv2d_img.shape}') 
plt.imshow(conv2d_img[4].reshape(14,14), cmap='Greys')  # [4] => filter 개수

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
img shape: (1, 28, 28, 1)
conv2d shape: (1, 14, 14, 50)
conv2d_img shape: (50, 14, 14, 1)


In [4]:
# tensorflow-MNIST with CNN
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# Data Loading
mnist=input_data.read_data_sets('./data/mnist', one_hot=True)


Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [13]:
# Graph 초기화
tf.reset_default_graph()

# placeholder
X=tf.placeholder(shape=[None,784], dtype=tf.float32)
Y=tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate=tf.placeholder(dtype=tf.float32)

# Convolution Layer  (img 개수, width, height, depth)
x_img=tf.reshape(X,[-1,28,28,1])

W1=tf.Variable(tf.random_normal([3,3,1,32]))  # 32 => CHANNEL
L1=tf.nn.conv2d(x_img, W1, strides=[1,1,1,1], padding='SAME')
L1=tf.nn.relu(L1)
# pooling
L1=tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob = keep_rate)

W2=tf.Variable(tf.random_normal([3,3,32,64]))
L2=tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2=tf.nn.relu(L2)
# pooling
L2=tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob = keep_rate)

# 이렇게 만든 데이터를 FC Layer에 넣어서 학습해야 함
L2=tf.reshape(L2,[-1,7*7*64])

W3=tf.get_variable('weight3', shape=[7*7*64,256],
                   initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.random_normal([256]), name='bias1')
_layer1=tf.nn.relu(tf.matmul(L2,W3)+b1)
layer1=tf.nn.dropout(_layer1,keep_prob=keep_rate)


W4=tf.get_variable('weight4',
                   shape=[256,10],
                   initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([10]), name='bias2')

# Hypothesis
logit=tf.matmul(layer1,W4)+b2
H=tf.nn.relu(logit)

# Cost
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                               labels=Y))

# train
train=tf.train.AdamOptimizer(learning_rate=.01).minimize(cost)

# session & initializer
sess=tf.Session()
sess.run(tf.global_variables_initializer())

# learning
cnt=0
num_of_epoch=30  # 반복 횟수
batch_size=100   # data size

for step in range(num_of_epoch):
    num_of_iter=int(mnist.train.num_examples/batch_size)  # 몇 번 반복?
    cost_val=0   # 초기화, 안해도 됨
    
    for i in range(num_of_iter):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        
        _, cost_val=sess.run([train, cost], feed_dict={X:batch_x,
                                                       Y:batch_y,
                                                       keep_rate:.7})
    
    if step%3==0:
        cnt+=1
        print(f'cost value{cnt}: {cost_val}')

cost value1: 0.6142646074295044
cost value2: 0.18215391039848328
cost value3: 0.6505834460258484
cost value4: 0.08644069731235504
cost value5: 0.125531867146492
cost value6: 0.15061932802200317
cost value7: 0.31346338987350464
cost value8: 0.2664753198623657
cost value9: 0.134915292263031
cost value10: 0.2140946388244629


In [14]:
# accuracy 측정
predict=tf.argmax(H,1)
correct=tf.equal(predict, tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

print(f'Accuracy: {sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels,keep_rate:1})}')

Accuracy: 0.9861000180244446


In [6]:
7*7*64

3136